In [ ]:
import time
import math
import numpy as np
import pandas as pd
import scipy as sc
from scipy.optimize import minimize
from scipy.optimize import Bounds

Download dataset

In [ ]:
# on Kaggle
# kaggle competitions download -c dmia-sport-2019-fall-intro

train_df = pd.read_csv('/content/train (1).csv')
test_df = pd.read_csv('/content/Xtest.csv')

Delete NaN values and refactor to string format

In [ ]:
train_df = train_df.replace({np.nan: ''})
train_df.Password = train_df.Password.apply(str)
train_df.astype({'Password': 'U20'})

test_df = test_df.replace({np.nan: ''})
test_df.Password = test_df.Password.apply(str)
test_df.astype({'Password': 'U20'})

Realize the new columns for training database

In [ ]:
def many_len_metrics(x):
    d = {'Спец.символы': 0, 'Буквы': 0, 'Цифры': 0}
    for i in x:
        if i.isalpha():
            d['Буквы'] += 1
        elif i.isdigit():
            d['Цифры'] += 1
        else:
            d['Спец.символы'] += 1

    return d['Цифры'], d['Буквы'], d['Спец.символы']

def len_password(x: str):
    return len(x)

def len_unique(x: str):
    return len(set(x))

def len_specific(x: str):
    return many_len_metrics(x)[-1]

def len_alfabet(x):
    return many_len_metrics(x)[1]

def len_digit(x):
    return many_len_metrics(x)[0]

def attitude_len_for_unique(x):
    if len_password(x) != 0:
      return len_unique(x) / len_password(x)
    return 0

def len_big_alfa(x):
    return sum(1 for i in x if i.isupper())

def len_mini_alfa(x):
    return sum(1 for i in x if i.islower())

def attitude_mini_for_big(x):
    if len_big_alfa(x) != 0:
      return len_mini_alfa(x) / len_big_alfa(x)
    return 0

def Entropy(x):
    f1, f2, f3, f4 = 0, 0, 0, 0
    for i in x:
        if i.isupper():
          f1 = 1;
        elif i.islower():
          f2 = 1
        elif i.isdigit():
          f3 = 1;
        else:
          f4 = 1
    ans = 1
    if f1:
      ans+=26
    if f2:
      ans+=26
    if f3:
      ans+=10
    if f4:
      ans+=22
    return len(x)*math.log2(ans)

Create columns for each function and add to DataFrame

In [ ]:
train_df['len_pass'] = train_df['Password'].apply(len_password)
train_df['len_unique'] = train_df['Password'].apply(len_unique)
train_df['len_specific'] = train_df['Password'].apply(len_specific)
train_df['len_alfabet'] = train_df['Password'].apply(len_alfabet)
train_df['len_digit'] = train_df['Password'].apply(len_digit)
train_df['attitude_len_for_unique'] = train_df['Password'].apply(attitude_len_for_unique)
train_df['len_big_alfa'] = train_df['Password'].apply(len_big_alfa)
train_df['len_mini_alfa'] = train_df['Password'].apply(len_mini_alfa)
train_df['attitude_mini_for_big'] = train_df['Password'].apply(attitude_mini_for_big)

train_df['entropy'] = train_df['Password'].apply(Entropy)

In [ ]:
# Create X and y who will split on train and test in future
features = list(train_df.iloc[:, 2:11])
X = train_df[features]

y = train_df['Times']
y = y.replace('', '0')
y = y.astype('int')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.8,
    random_state=42,
)

Initialize model and fit it

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
def save_submission(predict):
    sub = pd.DataFrame(columns=['Id', 'Times'])
    sub['Id'] = range(len(predict))
    sub = sub.set_index('Id')
    sub['Times'] = list(map(int, predict))
    with open(f'sub_{id(predict)}.csv', 'w') as f:
        f.write(sub.to_csv())

In [ ]:
save_submission(y_pred)